# Unsupervised Learning Trading Strategy

In [16]:
import numpy as np
import pandas as pd
import pandas_ta
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
import warnings
warnings.filterwarnings('ignore')


## Getting data

Getting the companies' symbols from Wiki.
Caution : not survivalship bias free

In [2]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500.head()


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
sp500.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


Some of the symbols have dots, we will replace those with dashes this will prevent eny error from API yfinance.

In [4]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')


In [5]:
# extracting s&p500 codes into a list
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list[:10]


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADM', 'ADBE', 'ADP', 'AES', 'AFL']

Now lets get historical data from yfinance API. Start date will be 28/11/23 and we will get 8 years of historical data.

In [6]:
end_date = '2023-11-18'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)
start_date


Timestamp('2015-11-20 00:00:00')

In [7]:
original_df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date)

original_df.head()


[*********************100%***********************]  503 of 503 completed


Adj Close                                        \
                                   A        AAL       AAPL       ABBV ABNB   
Date                                                                         
2015-11-20 00:00:00-05:00  36.774700  40.328243  27.152472  43.255482  NaN   
2015-11-23 00:00:00-05:00  37.476860  40.423801  26.799694  43.283794  NaN   
2015-11-24 00:00:00-05:00  38.076046  39.401268  27.056885  43.269634  NaN   
2015-11-25 00:00:00-05:00  38.703312  39.477711  26.863422  42.675053  NaN   
2015-11-27 00:00:00-05:00  39.293114  39.840858  26.813353  42.469788  NaN   

                                                                       \
                                 ABT       ACGL        ACN       ADBE   
Date                                                                    
2015-11-20 00:00:00-05:00  39.607716  24.686666  95.005859  91.809998   
2015-11-23 00:00:00-05:00  39.117043  24.643333  94.891487  91.959999   
2015-11-24 00:00:00-05:00  39.125645  24.533333  94.460297  92.000000   
2015-11-25 00:00:00-05:00  39.091217  24.216667  94.240334  91.769997   
2015-11-27 00:00:00-05:00  39.099819  24.306667  94.557098  92.169998   

                                      ...   Volume                             \
                                 ADI  ...     WYNN      XEL       XOM    XRAY   
Date                                  ...                                       
2015-11-20 00:00:00-05:00  50.314316  ...  3778300  3883400  10144400  880900   
2015-11-23 00:00:00-05:00  48.095818  ...  2427400  2341500  11868400  616000   
2015-11-24 00:00:00-05:00  51.161068  ...  5628300  4416300  15055500  917900   
2015-11-25 00:00:00-05:00  51.017117  ...  2358500  2200100   8980400  365000   
2015-11-27 00:00:00-05:00  51.186481  ...  2285100  1468000   4156600  293200   

                                                                                
                               XYL      YUM      ZBH    ZBRA     ZION      ZTS  
Date                                                                            
2015-11-20 00:00:00-05:00  1300900  5928998  1692187  303500  2092400  3679400  
2015-11-23 00:00:00-05:00   870600  3554005  1031648  320300  1438700  3265300  
2015-11-24 00:00:00-05:00  1044000  3045455  1313868  431800  1608600  1713500  
2015-11-25 00:00:00-05:00   916200  3222808  1008576  292100  1086200  2196800  
2015-11-27 00:00:00-05:00   514200  2558884   917730  175700   763200  1155100  

[5 rows x 3018 columns]

In [9]:
# getting rid of multi index
df = original_df.stack()
df


Adj Close       Close        High  \
Date                                                                 
2015-11-20 00:00:00-05:00 A      36.774700   39.279999   39.349998   
                          AAL    40.328243   42.200001   42.709999   
                          AAPL   27.152472   29.825001   29.980000   
                          ABBV   43.255482   61.110001   61.360001   
                          ABT    39.607716   46.009998   46.380001   
...                                    ...         ...         ...   
2023-11-17 00:00:00-05:00 YUM   127.660004  127.660004  128.490005   
                          ZBH   111.669998  111.669998  112.660004   
                          ZBRA  218.020004  218.020004  218.699997   
                          ZION   36.070000   36.070000   36.470001   
                          ZTS   174.800003  174.800003  177.699997   

                                       Low        Open       Volume  
Date                                                                 
2015-11-20 00:00:00-05:00 A      38.520000   38.779999    5447900.0  
                          AAL    42.049999   42.400002    5656000.0  
                          AAPL   29.712500   29.799999  137148400.0  
                          ABBV   60.570000   60.939999    7490200.0  
                          ABT    45.840000   46.029999    7140700.0  
...                                    ...         ...          ...  
2023-11-17 00:00:00-05:00 YUM   127.250000  128.419998    1089600.0  
                          ZBH   111.330002  112.349998    2991400.0  
                          ZBRA  215.270004  218.460007     340600.0  
                          ZION   35.400002   36.250000    2136200.0  
                          ZTS   173.699997  177.410004    2425100.0  

[994819 rows x 6 columns]

In [11]:
# renaming indexes
df.index.names = ['date','ticker']
df.head()


Adj Close      Close       High        Low  \
date                      ticker                                               
2015-11-20 00:00:00-05:00 A       36.774700  39.279999  39.349998  38.520000   
                          AAL     40.328243  42.200001  42.709999  42.049999   
                          AAPL    27.152472  29.825001  29.980000  29.712500   
                          ABBV    43.255482  61.110001  61.360001  60.570000   
                          ABT     39.607716  46.009998  46.380001  45.840000   

                                       Open       Volume  
date                      ticker                          
2015-11-20 00:00:00-05:00 A       38.779999    5447900.0  
                          AAL     42.400002    5656000.0  
                          AAPL    29.799999  137148400.0  
                          ABBV    60.939999    7490200.0  
                          ABT     46.029999    7140700.0

In [12]:
#renaming columns for more practical use
df.columns = df.columns.str.lower()
df.head()


adj close      close       high        low  \
date                      ticker                                               
2015-11-20 00:00:00-05:00 A       36.774700  39.279999  39.349998  38.520000   
                          AAL     40.328243  42.200001  42.709999  42.049999   
                          AAPL    27.152472  29.825001  29.980000  29.712500   
                          ABBV    43.255482  61.110001  61.360001  60.570000   
                          ABT     39.607716  46.009998  46.380001  45.840000   

                                       open       volume  
date                      ticker                          
2015-11-20 00:00:00-05:00 A       38.779999    5447900.0  
                          AAL     42.400002    5656000.0  
                          AAPL    29.799999  137148400.0  
                          ABBV    60.939999    7490200.0  
                          ABT     46.029999    7140700.0

In [13]:
#saving data for futher tests and avoiding being blocked
df.to_csv('raw_data/stocks_stack.csv')


In [5]:
# getting data from csv that would correspond to the transformed initial df
df_csv = pd.read_csv('raw_data/stocks_stack.csv')
df_csv.set_index(['date', 'ticker'], inplace=True)
df_csv


adj close       close        high  \
date                      ticker                                       
2015-11-20 00:00:00-05:00 A        36.774700   39.279999   39.349998   
                          AAL      40.328243   42.200001   42.709999   
                          AAPL     27.152472   29.825001   29.980000   
                          ABBV     43.255482   61.110001   61.360001   
                          ABT      39.607716   46.009998   46.380001   
...                                      ...         ...         ...   
2023-11-17 00:00:00-05:00 YUM     127.660004  127.660004  128.490005   
                          ZBH     111.669998  111.669998  112.660004   
                          ZBRA    218.020004  218.020004  218.699997   
                          ZION     36.070000   36.070000   36.470001   
                          ZTS     174.800003  174.800003  177.699997   

                                         low        open       volume  
date                      ticker                                       
2015-11-20 00:00:00-05:00 A        38.520000   38.779999    5447900.0  
                          AAL      42.049999   42.400002    5656000.0  
                          AAPL     29.712500   29.799999  137148400.0  
                          ABBV     60.570000   60.939999    7490200.0  
                          ABT      45.840000   46.029999    7140700.0  
...                                      ...         ...          ...  
2023-11-17 00:00:00-05:00 YUM     127.250000  128.419998    1089600.0  
                          ZBH     111.330002  112.349998    2991400.0  
                          ZBRA    215.270004  218.460007     340600.0  
                          ZION     35.400002   36.250000    2136200.0  
                          ZTS     173.699997  177.410004    2425100.0  

[994819 rows x 6 columns]

## Calculate features and technical indicators for each stock


* Garman-klass volatility
* RSI (Relative Strength Index)
* Bollinger Bands
* ATR
* MACD
* Dollar Volume

### Garman-klass volatility
This method provides a way to estimate future volatility based on historical price movements.

The formula is derived from the assumption that asset prices follow a log-normal distribution and that price changes are normally distributed.

$$ \sigma_{GK} = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N} \left(\log\left(\frac{H_i}{L_i}\right)^2 - \frac{2\log\left(\frac{C_i}{O_i}\right)\log\left(\frac{H_i}{L_i}\right)}{N} + \log\left(\frac{C_i}{O_i}\right)^2\right)}

* N is the number of data points i
* H<sub>i</sub>  is the high price of the asset at time i
* L<sub>i</sub> is the low price of the asset at time i
* O<sub>i</sub> is the open price of the asset at time i
* C<sub>i</sub> is the close price of the asset at time i


In [15]:
df_csv['garman_klass_vol'] = ((np.log(df_csv.high) - np.log(df_csv.low))**2)/2 - (2*np.log(2)-1)*(np.log(df_csv['adj close']) - np.log(df_csv.open))**2
df_csv


adj close       close        high  \
date                      ticker                                       
2015-11-20 00:00:00-05:00 A        36.774700   39.279999   39.349998   
                          AAL      40.328243   42.200001   42.709999   
                          AAPL     27.152472   29.825001   29.980000   
                          ABBV     43.255482   61.110001   61.360001   
                          ABT      39.607716   46.009998   46.380001   
...                                      ...         ...         ...   
2023-11-17 00:00:00-05:00 YUM     127.660004  127.660004  128.490005   
                          ZBH     111.669998  111.669998  112.660004   
                          ZBRA    218.020004  218.020004  218.699997   
                          ZION     36.070000   36.070000   36.470001   
                          ZTS     174.800003  174.800003  177.699997   

                                         low        open       volume  \
date                      ticker                                        
2015-11-20 00:00:00-05:00 A        38.520000   38.779999    5447900.0   
                          AAL      42.049999   42.400002    5656000.0   
                          AAPL     29.712500   29.799999  137148400.0   
                          ABBV     60.570000   60.939999    7490200.0   
                          ABT      45.840000   46.029999    7140700.0   
...                                      ...         ...          ...   
2023-11-17 00:00:00-05:00 YUM     127.250000  128.419998    1089600.0   
                          ZBH     111.330002  112.349998    2991400.0   
                          ZBRA    215.270004  218.460007     340600.0   
                          ZION     35.400002   36.250000    2136200.0   
                          ZTS     173.699997  177.410004    2425100.0   

                                  garman_klass_vol  
date                      ticker                    
2015-11-20 00:00:00-05:00 A              -0.000862  
                          AAL            -0.000848  
                          AAPL           -0.003304  
                          ABBV           -0.045301  
                          ABT            -0.008654  
...                                            ...  
2023-11-17 00:00:00-05:00 YUM             0.000033  
                          ZBH             0.000056  
                          ZBRA            0.000123  
                          ZION            0.000434  
                          ZTS             0.000174  

[994819 rows x 7 columns]

### Relative Strangth Index (RSI)

RSI stands for Relative Strength Index, and it's a popular momentum indicator used in trading to assess the magnitude of recent price changes and determine whether a stock or other asset is overbought or oversold.

RSI oscillates between 0 and 100 and is calculated based on the average gain and average loss over a specified period, typically **14** trading days. The formula involves comparing the average gains against the average losses over this period to calculate a relative strength.

Overbought and Oversold Conditions: When the RSI crosses **above 70**, it's considered **overbought**, suggesting that the asset may be due for a pullback or correction. Conversely, when the RSI drops **below 30**, it's considered **oversold**, indicating a potential upward reversal.

In [19]:
df_csv['rsi'] = df_csv.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=14))
df_csv


adj close       close        high  \
date                      ticker                                       
2015-11-20 00:00:00-05:00 A        36.774700   39.279999   39.349998   
                          AAL      40.328243   42.200001   42.709999   
                          AAPL     27.152472   29.825001   29.980000   
                          ABBV     43.255482   61.110001   61.360001   
                          ABT      39.607716   46.009998   46.380001   
...                                      ...         ...         ...   
2023-11-17 00:00:00-05:00 YUM     127.660004  127.660004  128.490005   
                          ZBH     111.669998  111.669998  112.660004   
                          ZBRA    218.020004  218.020004  218.699997   
                          ZION     36.070000   36.070000   36.470001   
                          ZTS     174.800003  174.800003  177.699997   

                                         low        open       volume  \
date                      ticker                                        
2015-11-20 00:00:00-05:00 A        38.520000   38.779999    5447900.0   
                          AAL      42.049999   42.400002    5656000.0   
                          AAPL     29.712500   29.799999  137148400.0   
                          ABBV     60.570000   60.939999    7490200.0   
                          ABT      45.840000   46.029999    7140700.0   
...                                      ...         ...          ...   
2023-11-17 00:00:00-05:00 YUM     127.250000  128.419998    1089600.0   
                          ZBH     111.330002  112.349998    2991400.0   
                          ZBRA    215.270004  218.460007     340600.0   
                          ZION     35.400002   36.250000    2136200.0   
                          ZTS     173.699997  177.410004    2425100.0   

                                  garman_klass_vol        rsi  
date                      ticker                               
2015-11-20 00:00:00-05:00 A              -0.000862        NaN  
                          AAL            -0.000848        NaN  
                          AAPL           -0.003304        NaN  
                          ABBV           -0.045301        NaN  
                          ABT            -0.008654        NaN  
...                                            ...        ...  
2023-11-17 00:00:00-05:00 YUM             0.000033  62.631823  
                          ZBH             0.000056  60.522122  
                          ZBRA            0.000123  54.353061  
                          ZION            0.000434  62.590903  
                          ZTS             0.000174  59.645233  

[994819 rows x 8 columns]

In [40]:
# 10 stocks most frequently overbought
df_csv[df_csv.rsi > 70][['rsi']].groupby(level=1).count().sort_values(by='rsi', ascending=False).head(10)


,rsi
ticker,
AMCR,326
CPRT,312
NVDA,305
AAPL,292
WM,284
WST,272
ADBE,268
LLY,266
CDNS,265


In [39]:
# 10 stocks most frequently oversold
df_csv[df_csv.rsi < 30][['rsi']].groupby(level=1).count().sort_values(by='rsi', ascending=False).head(10)


,rsi
ticker,
AMCR,268
ALK,183
KHC,169
VTRS,163
BBWI,159
PARA,151
SLB,140
WBA,139
GE,136
